In [1]:
import torch
from torch import nn
import env.torch as d2l

In [9]:
def corr2d(x,K):
    h,w=K.shape

    Y=torch.zeros(x.shape[0]-h+1,x.shape[1]-w+1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i,j]=(x[i:i+h,j:j+w]*K).sum()

    return Y

In [11]:
X=torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K=torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

卷积层

In [12]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight=nn.Parameter(torch.randn(kernel_size))
        self.bias=nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight)+self.bias

图像目标边缘检测

In [14]:
X=torch.ones((6,8))
X[:,2:6]=0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

构造高为 1，宽为 2 的卷积核K

In [15]:
K=torch.tensor([[1.0,-1.0]])

In [16]:
Y=corr2d(X,K)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [17]:
corr2d(X.t(),K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [18]:
#构造二维卷积层，具有一个输出通道和形状为（1,2）的卷积核
conv2d=nn.Conv2d(1,1,kernel_size=(1,2),bias=False)

X=X.reshape((1,1,6,8))
Y=Y.reshape((1,1,6,7))
Ir=3e-2  #学习率

for i in range(10):
    Y_hat=conv2d(X)
    I=(Y_hat-Y)**2
    conv2d.zero_grad()
    I.sum().backward()
    #迭代卷积核
    conv2d.weight.data[:]-=Ir*conv2d.weight.grad
    if(i+1)%2==0:
        print(f'epoch{i+1},loss{I.sum():.3f}')

epoch2,loss7.973
epoch4,loss1.353
epoch6,loss0.233
epoch8,loss0.042
epoch10,loss0.008


In [19]:
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9810, -0.9898]])